In [1]:
import pandas as pd
import numpy as np

In [2]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# plt.style.use('ggplot')

In [3]:
df = pd.read_csv('data/clean_final.csv',parse_dates=['date'])
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [4]:
df.sort_values(by=['date'],inplace=True)
df.reset_index(drop=True,inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116976 entries, 0 to 116975
Data columns (total 7 columns):
date                   116976 non-null datetime64[ns]
hour                   116976 non-null int64
beat                   116976 non-null object
premise                116976 non-null object
offense_type           116976 non-null object
offenses               116976 non-null int64
full_street_address    116976 non-null object
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 6.2+ MB


In [6]:
df.head()

,date,hour,beat,premise,offense_type,offenses,full_street_address
0,2017-01-01,12,18F20,Department or Discount Store,Theft,1,5150 WESTHEIMER RD
1,2017-01-01,22,8C60,Residence or House,Aggravated Assault,1,11150 DALEBROOK DR
2,2017-01-01,17,20G30,Service or Gas Station,Robbery,1,2750 HOUSTON PKWY
3,2017-01-01,16,14D40,Apartment,Robbery,1,12050 KING BLVD
4,2017-01-01,17,14D40,Residence or House,Aggravated Assault,1,4350 HARTSVILLE RD


# month crime rate

In [7]:
df.date.dt.strftime('%b').value_counts()

May    10369
Jan    10314
Jul    10300
Jun    10209
Aug     9921
Dec     9765
Mar     9653
Oct     9587
Feb     9552
Nov     9510
Apr     9348
Sep     8448
Name: date, dtype: int64

## select september

In [25]:
sep = df.date.dt.strftime('%b') =='Sep'

In [26]:
sepdf = df[sep]

In [29]:
sepdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8448 entries, 79666 to 88113
Data columns (total 7 columns):
date                   8448 non-null datetime64[ns]
hour                   8448 non-null int64
beat                   8448 non-null object
premise                8448 non-null object
offense_type           8448 non-null object
offenses               8448 non-null int64
full_street_address    8448 non-null object
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 528.0+ KB


In [34]:
sepdf.reset_index(inplace=True)

In [35]:
sepdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8448 entries, 0 to 8447
Data columns (total 8 columns):
index                  8448 non-null int64
date                   8448 non-null datetime64[ns]
hour                   8448 non-null int64
beat                   8448 non-null object
premise                8448 non-null object
offense_type           8448 non-null object
offenses               8448 non-null int64
full_street_address    8448 non-null object
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 528.1+ KB


In [36]:
sepdf.head()

,index,date,hour,beat,premise,offense_type,offenses,full_street_address
0,79666,2017-09-01,7,24C50,Hospital,Theft,1,23050 EASTEX FWY IB
1,79667,2017-09-01,9,11H20,Convenience Store Parking Lot,Theft,1,8050 ELVERA ST
2,79668,2017-09-01,10,1A10,"Hotel, Motel, Inn, Etc.",Theft,1,550 DALLAS ST
3,79669,2017-09-01,22,1A10,Convention Center or Exhibit Halls,Theft,2,1050 AVENIDA DE LAS AMERI
4,79670,2017-09-01,8,11H20,"Road, Street, or Sidewalk",Theft,1,650 RANKIN RD


## get lat_lot

In [41]:
#! pip install geopy

In [1]:
from geopy.geocoders import Nominatim
from tqdm import tqdm
tqdm.pandas()

In [44]:
test = sepdf.loc[:150].copy()  # test dataset

In [46]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 8 columns):
index                  151 non-null int64
date                   151 non-null datetime64[ns]
hour                   151 non-null int64
beat                   151 non-null object
premise                151 non-null object
offense_type           151 non-null object
offenses               151 non-null int64
full_street_address    151 non-null object
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 9.5+ KB


In [47]:
test.head()

,index,date,hour,beat,premise,offense_type,offenses,full_street_address
0,79666,2017-09-01,7,24C50,Hospital,Theft,1,23050 EASTEX FWY IB
1,79667,2017-09-01,9,11H20,Convenience Store Parking Lot,Theft,1,8050 ELVERA ST
2,79668,2017-09-01,10,1A10,"Hotel, Motel, Inn, Etc.",Theft,1,550 DALLAS ST
3,79669,2017-09-01,22,1A10,Convention Center or Exhibit Halls,Theft,2,1050 AVENIDA DE LAS AMERI
4,79670,2017-09-01,8,11H20,"Road, Street, or Sidewalk",Theft,1,650 RANKIN RD


In [48]:
geolocator = Nominatim(user_agent="specify_your_app_name_here",
                       format_string="%s, Houston, TX")

from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
test['location'] = test['full_street_address'].progress_apply(geocode)

100%|██████████| 151/151 [04:21<00:00,  1.75s/it]


In [49]:
test.head()

,index,date,hour,beat,premise,offense_type,offenses,full_street_address,location
0,79666,2017-09-01,7,24C50,Hospital,Theft,1,23050 EASTEX FWY IB,None
1,79667,2017-09-01,9,11H20,Convenience Store Parking Lot,Theft,1,8050 ELVERA ST,"(8050, Elvera Street, Houston, Harris County, ..."
2,79668,2017-09-01,10,1A10,"Hotel, Motel, Inn, Etc.",Theft,1,550 DALLAS ST,"(550, Dallas Street, Houston, Harris County, T..."
3,79669,2017-09-01,22,1A10,Convention Center or Exhibit Halls,Theft,2,1050 AVENIDA DE LAS AMERI,None
4,79670,2017-09-01,8,11H20,"Road, Street, or Sidewalk",Theft,1,650 RANKIN RD,"(Rankin Road, North Houston District, Houston,..."


In [50]:
%%time
test['address'] = test['location'].apply(lambda loc: loc.address if loc else None)
test['lat_lon'] = test['location'].apply(lambda loc: tuple(loc.point[:2]) if loc else None)

CPU times: user 19.5 ms, sys: 0 ns, total: 19.5 ms
Wall time: 19.3 ms


In [54]:
def correct(row):
    '''checks if addres is good by checking 
    if col address contains a street number
    returns bad if  not'''
    if row['address']!= None:
        a = row['address'].split(',')[0] # address number
        if a.isdigit():
            return 'good'
        else:
            return 'bad'

In [55]:
test['good_bad'] = test.progress_apply(correct,axis=1)

100%|██████████| 151/151 [00:00<00:00, 6942.84it/s]


In [57]:
test.good_bad.value_counts(dropna=False)

good    86
bad     42
NaN     23
Name: good_bad, dtype: int64

### Test result
- total test address: 150
- 86 were good
- 42 were bad: no street number return from geopy
- 23 were nan: no possible address found

## try september dataframe


In [58]:
sepdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8448 entries, 0 to 8447
Data columns (total 8 columns):
index                  8448 non-null int64
date                   8448 non-null datetime64[ns]
hour                   8448 non-null int64
beat                   8448 non-null object
premise                8448 non-null object
offense_type           8448 non-null object
offenses               8448 non-null int64
full_street_address    8448 non-null object
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 528.1+ KB


In [64]:
df.head()

,date,hour,beat,premise,offense_type,offenses,full_street_address
0,2017-01-01,12,18F20,Department or Discount Store,Theft,1,5150 WESTHEIMER RD
1,2017-01-01,22,8C60,Residence or House,Aggravated Assault,1,11150 DALEBROOK DR
2,2017-01-01,17,20G30,Service or Gas Station,Robbery,1,2750 HOUSTON PKWY
3,2017-01-01,16,14D40,Apartment,Robbery,1,12050 KING BLVD
4,2017-01-01,17,14D40,Residence or House,Aggravated Assault,1,4350 HARTSVILLE RD


In [66]:
top_crime = df.beat =='1A20'

In [68]:
tcdf = df[top_crime]

In [69]:
tcdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2761 entries, 152 to 116846
Data columns (total 7 columns):
date                   2761 non-null datetime64[ns]
hour                   2761 non-null int64
beat                   2761 non-null object
premise                2761 non-null object
offense_type           2761 non-null object
offenses               2761 non-null int64
full_street_address    2761 non-null object
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 172.6+ KB


In [70]:
tcdf.head()

,date,hour,beat,premise,offense_type,offenses,full_street_address
152,2017-01-01,12,1A20,Bar or Night Club,Aggravated Assault,1,650 PARK BLVD
153,2017-01-01,0,1A20,unknown,Aggravated Assault,1,2350 SMITH
154,2017-01-01,13,1A20,"Road, Street, or Sidewalk",Theft,1,450 MARSHALL
155,2017-01-01,13,1A20,"Road, Street, or Sidewalk",Theft,1,250 AVONDALE ST
526,2017-01-02,21,1A20,"Road, Street, or Sidewalk",Theft,1,3150 ALLEN PKWY


## start 17:07
## end:

In [71]:
geolocator = Nominatim(user_agent="specify_your_app_name_here",
                       format_string="%s, Houston, TX")

from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
tcdf['location'] = tcdf['full_street_address'].progress_apply(geocode)


100%|██████████| 2761/2761 [1:19:29<00:00,  1.71s/it]
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [72]:
tcdf.to_csv('data/tcdf.csv')

In [73]:
tcdf.head()

,date,hour,beat,premise,offense_type,offenses,full_street_address,location
152,2017-01-01,12,1A20,Bar or Night Club,Aggravated Assault,1,650 PARK BLVD,None
153,2017-01-01,0,1A20,unknown,Aggravated Assault,1,2350 SMITH,"(2350, Smith Street, Houston, Harris County, T..."
154,2017-01-01,13,1A20,"Road, Street, or Sidewalk",Theft,1,450 MARSHALL,"(450, Marshall Street, Houston, Harris County,..."
155,2017-01-01,13,1A20,"Road, Street, or Sidewalk",Theft,1,250 AVONDALE ST,"(250, Avondale Street, Houston, Harris County,..."
526,2017-01-02,21,1A20,"Road, Street, or Sidewalk",Theft,1,3150 ALLEN PKWY,"(3150, Allen Parkway, Houston, Harris County, ..."


In [74]:
def correct(row):
    '''checks if addres is good by checking 
    if col address contains a street number
    returns bad if  not'''
    if row['address']!= None:
        a = row['address'].split(',')[0] # address number
        if a.isdigit():
            return 'good'
        else:
            return 'bad'

In [75]:
tcdf['address'] = tcdf['location'].apply(lambda loc: loc.address if loc else None)
tcdf['lat_lon'] = tcdf['location'].apply(lambda loc: tuple(loc.point[:2]) if loc else None)

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [76]:
tcdf.head()

,date,hour,beat,premise,offense_type,offenses,full_street_address,location,address,lat_lon
152,2017-01-01,12,1A20,Bar or Night Club,Aggravated Assault,1,650 PARK BLVD,None,None,None
153,2017-01-01,0,1A20,unknown,Aggravated Assault,1,2350 SMITH,"(2350, Smith Street, Houston, Harris County, T...","2350, Smith Street, Houston, Harris County, Te...","(29.7490344285714, -95.3764075102041)"
154,2017-01-01,13,1A20,"Road, Street, or Sidewalk",Theft,1,450 MARSHALL,"(450, Marshall Street, Houston, Harris County,...","450, Marshall Street, Houston, Harris County, ...","(29.7397129387755, -95.3829442244898)"
155,2017-01-01,13,1A20,"Road, Street, or Sidewalk",Theft,1,250 AVONDALE ST,"(250, Avondale Street, Houston, Harris County,...","250, Avondale Street, Houston, Harris County, ...","(29.7454171020408, -95.3839227142857)"
526,2017-01-02,21,1A20,"Road, Street, or Sidewalk",Theft,1,3150 ALLEN PKWY,"(3150, Allen Parkway, Houston, Harris County, ...","3150, Allen Parkway, Houston, Harris County, T...","(29.7608958711171, -95.400778054397)"


In [77]:
tcdf['good_bad'] = tcdf.progress_apply(correct,axis=1)


100%|██████████| 2761/2761 [00:00<00:00, 5146.56it/s]/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [78]:
tcdf.head()

,date,hour,beat,premise,offense_type,offenses,full_street_address,location,address,lat_lon,good_bad
152,2017-01-01,12,1A20,Bar or Night Club,Aggravated Assault,1,650 PARK BLVD,None,None,None,None
153,2017-01-01,0,1A20,unknown,Aggravated Assault,1,2350 SMITH,"(2350, Smith Street, Houston, Harris County, T...","2350, Smith Street, Houston, Harris County, Te...","(29.7490344285714, -95.3764075102041)",good
154,2017-01-01,13,1A20,"Road, Street, or Sidewalk",Theft,1,450 MARSHALL,"(450, Marshall Street, Houston, Harris County,...","450, Marshall Street, Houston, Harris County, ...","(29.7397129387755, -95.3829442244898)",good
155,2017-01-01,13,1A20,"Road, Street, or Sidewalk",Theft,1,250 AVONDALE ST,"(250, Avondale Street, Houston, Harris County,...","250, Avondale Street, Houston, Harris County, ...","(29.7454171020408, -95.3839227142857)",good
526,2017-01-02,21,1A20,"Road, Street, or Sidewalk",Theft,1,3150 ALLEN PKWY,"(3150, Allen Parkway, Houston, Harris County, ...","3150, Allen Parkway, Houston, Harris County, T...","(29.7608958711171, -95.400778054397)",good


In [79]:
tcdf.good_bad.value_counts(dropna=False)

good    2074
bad      403
NaN      284
Name: good_bad, dtype: int64

In [87]:
tcdf.address.loc[153]

'2350, Smith Street, Houston, Harris County, Texas, 77006, USA'

In [88]:
tcdf.drop(['location'],axis=1,inplace=True)

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [89]:
tcdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2761 entries, 152 to 116846
Data columns (total 10 columns):
date                   2761 non-null datetime64[ns]
hour                   2761 non-null int64
beat                   2761 non-null object
premise                2761 non-null object
offense_type           2761 non-null object
offenses               2761 non-null int64
full_street_address    2761 non-null object
address                2477 non-null object
lat_lon                2477 non-null object
good_bad               2477 non-null object
dtypes: datetime64[ns](1), int64(2), object(7)
memory usage: 317.3+ KB


In [90]:
tcdf.head()

,date,hour,beat,premise,offense_type,offenses,full_street_address,address,lat_lon,good_bad
152,2017-01-01,12,1A20,Bar or Night Club,Aggravated Assault,1,650 PARK BLVD,None,None,None
153,2017-01-01,0,1A20,unknown,Aggravated Assault,1,2350 SMITH,"2350, Smith Street, Houston, Harris County, Te...","(29.7490344285714, -95.3764075102041)",good
154,2017-01-01,13,1A20,"Road, Street, or Sidewalk",Theft,1,450 MARSHALL,"450, Marshall Street, Houston, Harris County, ...","(29.7397129387755, -95.3829442244898)",good
155,2017-01-01,13,1A20,"Road, Street, or Sidewalk",Theft,1,250 AVONDALE ST,"250, Avondale Street, Houston, Harris County, ...","(29.7454171020408, -95.3839227142857)",good
526,2017-01-02,21,1A20,"Road, Street, or Sidewalk",Theft,1,3150 ALLEN PKWY,"3150, Allen Parkway, Houston, Harris County, T...","(29.7608958711171, -95.400778054397)",good


In [91]:
tcdf.to_csv('data/tcdf.csv')